In [ ]:
import sys
import os
import json
import glob
import h5py
import numpy as np
import scipy.signal
import scipy.io.wavfile
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_figures
import util_stimuli

import bez2018model


In [ ]:
sr = 32000
t = np.arange(0, 0.050, 1/sr)
signal = np.zeros_like(t)
noise = np.random.randn(signal.shape[0])
f0 = 200

for freq in np.arange(f0, sr/2, f0):
    signal = signal + np.sin(2*np.pi*freq*t)

snr = 10
dBSPL = 60
noise = np.random.randn(noise.shape[0])

y = util_stimuli.combine_signal_and_noise(signal, noise, snr=snr)
y = util_stimuli.set_dBSPL(y, dBSPL)


figsize = (12, 2)
nrows = 1
ncols = 1
gridspec_kw = {}
fig, ax_arr = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, gridspec_kw=gridspec_kw)
util_figures.make_stimulus_summary_plot(ax_arr,
                                        ax_idx_waveform=0,
                                        waveform=y,
                                        sr_waveform=sr)
plt.tight_layout()
plt.show()

ipd.display(ipd.Audio(y, rate=sr))


In [ ]:
kwargs_nervegram_meanrates = {
    'meanrates_params': {
        'dur': 0.050,
        'fs': 20e3,
        'buffer_start_dur': 0.0,
        'buffer_end_dur': 0.0,
    },
    'ANmodel_params': {
        'num_cfs': 100,
        'min_cf': 125,
        'max_cf': 14e3,
        'spont_list': [70],
        'species': 2,
        'cihc': 1.0,
        'cohc': 1.0,
        'bandwidth_scale_factor': 1.0,
        'IhcLowPass_cutoff': 3000.0,
        'IhcLowPass_order': 7,
    },
}

out_dict = bez2018model.nervegram_meanrates(y, sr, **kwargs_nervegram_meanrates)

for k in sorted(out_dict.keys()):
    print(k, np.array(out_dict[k]).shape)


In [ ]:
figsize=(12, 6)
nrows=2
ncols=3
gridspec_kw = {
    'wspace': 0.15,
    'hspace': 0.15,
    'width_ratios': [1, 8, 1],
    'height_ratios': [1, 4],
}

waveform = out_dict['signal']
sr_waveform = out_dict['signal_fs']
nervegram = out_dict['meanrates']
sr_nervegram = out_dict['meanrates_fs']

fig, ax_arr = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, gridspec_kw=gridspec_kw)
util_figures.make_stimulus_summary_plot(ax_arr,
                                        ax_idx_waveform=1,
                                        ax_idx_spectrum=3,
                                        ax_idx_nervegram=4,
                                        ax_idx_excitation=5,
                                        waveform=waveform,
                                        nervegram=nervegram,
                                        sr_waveform=sr_waveform,
                                        sr_nervegram=sr_nervegram,
                                        cfs=out_dict['cf_list'])
plt.tight_layout()
plt.show()


In [ ]:
sr = 32000
t = np.arange(0, 0.100, 1/sr)

f1 = 500 # Probe
f2 = 600 # Suppressor
y1 = util_stimuli.set_dBSPL(np.sin(2*np.pi*f1*t + 2*np.pi * np.random.rand()), 60)
y2 = util_stimuli.set_dBSPL(np.sin(2*np.pi*f2*t + 2*np.pi * np.random.rand()), 60)
y = y1 + y2

for waveform in [y1, y2, y]:
    ipd.display(ipd.Audio(waveform, rate=sr))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 2))
    util_figures.make_stimulus_summary_plot(ax,
                                            ax_idx_waveform=0,
                                            waveform=waveform,
                                            sr_waveform=sr)
    plt.show()


In [ ]:
out_dict_y1 = bez2018model.nervegram_meanrates(y1, sr, **kwargs_nervegram_meanrates)
out_dict_y = bez2018model.nervegram_meanrates(y, sr, **kwargs_nervegram_meanrates)


In [ ]:
figsize=(12, 6)
nrows=2
ncols=3
gridspec_kw = {
    'wspace': 0.15,
    'hspace': 0.15,
    'width_ratios': [1, 8, 1],
    'height_ratios': [1, 4],
}

for out_dict in [out_dict_y1, out_dict_y]:    
    waveform = out_dict['signal']
    sr_waveform = out_dict['signal_fs']
    nervegram = out_dict['meanrates']
    sr_nervegram = out_dict['meanrates_fs']

    fig, ax_arr = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, gridspec_kw=gridspec_kw)
    util_figures.make_stimulus_summary_plot(ax_arr,
                                            ax_idx_waveform=1,
                                            ax_idx_spectrum=3,
                                            ax_idx_nervegram=4,
                                            ax_idx_excitation=5,
                                            waveform=waveform,
                                            nervegram=nervegram,
                                            sr_waveform=sr_waveform,
                                            sr_nervegram=sr_nervegram,
                                            cfs=out_dict['cf_list'])
    plt.show()


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 8))

color_list = ['k', 'r']
for cidx, out_dict in enumerate([out_dict_y1, out_dict_y]):
    util_figures.make_stimulus_summary_plot(ax,
        ax_idx_nervegram=None,
        ax_idx_spectrum=None,
        ax_idx_excitation=0,
        ax_idx_waveform=None,
        nervegram=out_dict['meanrates'],
        sr_nervegram=out_dict['meanrates_fs'],
        waveform=out_dict['pin'],
        sr_waveform=out_dict['pin_fs'],
        cfs=out_dict['cf_list'],
        limits_buffer=0.2,
        nxticks=6, 
        nyticks=6,
        spines_to_hide_spectrum=[],
        spines_to_hide_excitation=[],
        spines_to_hide_waveform=[])#['top', 'bottom', 'left', 'right'])
    ax.lines[-1].set(color=color_list[cidx], lw=2)
xticks = np.arange(0, 701, 100)
ax_arr.set_xticks(xticks)
ax_arr.set_xticklabels(xticks)
ax_arr.set_xlim([xticks[0], xticks[-1]])
ax_arr.set_xlabel('Mean firing rate (Hz)')
ax_arr.set_ylabel('Auditory nerve fibers')
plt.show()
